In [1]:
!git clone https://github.com/YoongiKim/CIFAR-10-images

Cloning into 'CIFAR-10-images'...
remote: Enumerating objects: 60027, done.
remote: Total 60027 (delta 0), reused 0 (delta 0), pack-reused 60027
Receiving objects: 100% (60027/60027), 19.94 MiB | 70.17 MiB/s, done.
Resolving deltas: 100% (59990/59990), done.
Checking out files: 100% (60001/60001), done.


In [2]:

!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [3]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import datasets,transforms
import torch.nn as nn
import torch
import cv2


In [4]:
#CUDA activation
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')


CUDA is available!  Training on GPU ...


In [5]:
#Creating the csv files
import os
p=[]
c=[]

for file_name in os.listdir("CIFAR-10-images/test/"):
  for files in os.listdir("CIFAR-10-images/test/"+file_name):
    if files.split(".")[-1].lower() in {"jpeg", "jpg", "png"}:
        path='CIFAR-10-images/test/'+file_name+'/'+files
        clss=file_name
        p.append(path)
        c.append(clss)

p1=[]
c1=[]

for file_name in os.listdir("CIFAR-10-images/train/"):
  for files in os.listdir("CIFAR-10-images/train/"+file_name):
    if files.split(".")[-1].lower() in {"jpeg", "jpg", "png"}:
        path='CIFAR-10-images/train/'+file_name+'/'+files
        clss=file_name
        p1.append(path)
        c1.append(clss)



test_dataset=pd.DataFrame({'path':p,'class':c})
train_dataset=pd.DataFrame({'path':p1,'class':c1})

test_d = test_dataset.to_csv('test_data.csv', index=False)
train_d = train_dataset.to_csv('train_data.csv', index=False)

In [7]:
# 3: defining data transformations

def data_transform(x):
  transform=transforms.Compose([transforms.RandomRotation(25),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5,0.5,0.5],
                                                     [0.5,0.5,0.5])])
  return transform(x)


In [13]:
## Step4 : Writting a custom Dataloader
class MyDataset():
  def __init__(self,image_set,dict1,argument=True):
    csv_reader = pd.read_csv(image_set)
    self.imgfiles=list(csv_reader.iloc[:,0])
    self.classlabels= list(csv_reader.iloc[:,1])
    self.argument=argument
    self.dict1=dict1
  def __len__(self):
    return len(self.imgfiles)
  def __getitem__(self,idx):
    img=cv2.imread(self.imgfiles[idx])
    if self.argument:
      x = Image.fromarray(img)
      X=data_transform(x)
    Y=self.dict1[self.classlabels[idx]]
    return X,Y

In [9]:
c_dict={'airplane':0,'automobile':1,'bird':2,'cat':3,'deer':4,'dog':5
        ,'frog':6,'horse':7,'ship':8,'truck':9}

In [15]:
from torch.utils.data.sampler import SubsetRandomSampler

In [18]:
## Preparing the iterators:
def prep_iter():
  c_dict={'airplane':0,'automobile':1,'bird':2,'cat':3,'deer':4,'dog':5
        ,'frog':6,'horse':7,'ship':8,'truck':9}

  train_data=MyDataset('train_data.csv',c_dict)
  test_data=MyDataset('test_data.csv',c_dict,argument=False)

  batch_size=20
  num_workers=0
  valid_size=0.2

  # obtain training indices that will be used for validation
  num_train = len(train_data) 
  indices = list(range(num_train))
  np.random.shuffle(indices)
  split = int(np.floor(valid_size * num_train))
  train_idx, valid_idx = indices[split:], indices[:split]

  # define samplers for obtaining training and validation batches
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
      sampler=train_sampler, num_workers=num_workers)
  valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
      sampler=valid_sampler, num_workers=num_workers)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
      num_workers=num_workers)

  return train_loader,valid_loader,test_loader

In [19]:
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1=nn.Conv2d(3,16,3,padding=1)
    self.conv2=nn.Conv2d(16,32,3,padding=1)
    self.conv3=nn.Conv2d(32,64,3,padding=1)
    self.pool=nn.MaxPool2d(2,2)
    self.fc1=nn.Linear(64*4*4,500)
    self.fc2=nn.Linear(500,10)
    self.dropout=nn.Dropout(0.25)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x))) 
    x=self.pool(F.relu(self.conv3(x)))
    x=x.view(-1,64*4*4)  #flattening
    x=self.dropout(x) 
    x=F.relu(self.fc1(x))
    x=self.dropout(x)
    x=F.relu(self.fc2(x))
    
    return x

In [20]:


def train(n_epochs,train_loader,valid_loader):
  # create a complete CNN
  model = CNN()
  #print(model)

  # move tensors to GPU if CUDA is available
  if train_on_gpu:
      model.cuda()    
  # specify loss function (categorical cross-entropy)
  criterion = nn.CrossEntropyLoss()

  # specify optimizer
  optimizer = optim.SGD(model.parameters(), lr=0.01)
  
  valid_loss_min = np.Inf # track change in validation loss
  t_loss=[]
  v_loss=[]
  for epoch in range(1, n_epochs+1):

      # keep track of training and validation loss
      train_loss = 0.0
      valid_loss = 0.0
    
      ###################
      # train the model #
      ###################
      model.train()
      for data, target in train_loader:
          #target=torch.Tensor(target)
          # move tensors to GPU if CUDA is available
          if train_on_gpu:
              data = data.cuda()
              target=target.cuda()
          # clear the gradients of all optimized variables
          optimizer.zero_grad()
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(data)
          # calculate the batch loss
          loss = criterion(output, target)
          # backward pass: compute gradient of the loss with respect to model parameters
          loss.backward()
          # perform a single optimization step (parameter update)
          optimizer.step()
          # update training loss
          train_loss += loss.item()*data.size(0)
        
      ######################    
      # validate the model #
      ######################
      model.eval()
      for data, target in valid_loader:
          # move tensors to GPU if CUDA is available
          if train_on_gpu:
              data, target = data.cuda(), target.cuda()
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(data)
          top_p,top_class=output.topk(1,dim=1)
          equalstry=top_class==target
          equals=top_class==target.view(*top_class.shape)
          accuracy=torch.mean(equals.type(torch.FloatTensor))
          # calculate the batch loss
          loss = criterion(output, target)
          # update average validation loss 
          valid_loss += loss.item()*data.size(0)
    
      # calculate average losses
      train_loss = train_loss/len(train_loader.sampler)
      valid_loss = valid_loss/len(valid_loader.sampler)

      t_loss.append(train_loss)
      v_loss.append(valid_loss) 

      # print training/validation statistics 
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss, valid_loss))
    
      # save model if validation loss has decreased
      if valid_loss <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min,
          valid_loss))
          torch.save(model.state_dict(), 'model_cifar.pt')
          valid_loss_min = valid_loss
          print(f'Accuracy: {accuracy.item()*100}%')

In [23]:
from torch import optim

In [24]:
train(30,train_loader,valid_loader)

NameError: ignored